#### File Manager 

In [2]:
import os
import json
import boto3
from botocore.exceptions import ClientError
from datetime import datetime
from botocore.exceptions import NoCredentialsError
from dotenv import load_dotenv
load_dotenv()

class FileManager:
    def __init__(self, aws_access_key, aws_secret_key, region_name, bucket_name):
        self.s3 = boto3.client(
            's3',
            aws_access_key_id=aws_access_key,
            aws_secret_access_key=aws_secret_key,
            region_name=region_name
        )
        self.bucket_name = bucket_name

    def read_json(self, s3_key):
        response = self.s3.get_object(Bucket=self.bucket_name, Key=s3_key)
        content = response['Body'].read().decode('utf-8')
        return json.loads(content)

    def read_txt_file(self, s3_key):
        response = self.s3.get_object(Bucket=self.bucket_name, Key=s3_key)
        content = response['Body'].read().decode('utf-8')
        return content

    def write_json(self, s3_key, data):
        json_data = json.dumps(data, default=str, indent=4)
        self.s3.put_object(
            Bucket=self.bucket_name,
            Key=s3_key,
            Body=json_data,
            ContentType="application/json"
        )
        print(f"Metadata written to s3://{self.bucket_name}/{s3_key}")

    def check_if_file_exists(self, s3_key):
        try:
            self.s3.head_object(Bucket=self.bucket_name, Key=s3_key)
            return True
        except ClientError as e:
            # Check if the error is specifically about the object not existing
            if e.response['Error']['Code'] == '404':
                return False
            else:
                # Re-raise for other errors
                raise

    def upload_file(self, obj, s3_key):
        if isinstance(obj, dict):
            body = json.dumps(obj, default=str, indent=4).encode('utf-8')
            content_type = "application/json"
        elif isinstance(obj, str):
            body = obj.encode('utf-8')
            content_type = "text/plain"
        else:
            raise ValueError("Unsupported file type")
        self.s3.put_object(
            Bucket=self.bucket_name,
            Key=s3_key,
            Body=body,
            ContentType=content_type
        )
        print(f"File uploaded to s3://{self.bucket_name}/{s3_key}")

    def upload_audio_file(self, audio_path, s3_key):
        with open(audio_path, 'rb') as f:
            audio_data = f.read()
        self.s3.put_object(
            Bucket=self.bucket_name,
            Key=s3_key,
            Body=audio_data,
            ContentType="audio/mpeg"
        )
        os.remove(audio_path)
        print(f"Audio file uploaded to s3://{self.bucket_name}/{s3_key} and local file deleted.")

    def download_file(self, s3_key, local_path):
        self.s3.download_file(self.bucket_name, s3_key, local_path)
        print(f"File downloaded from s3://{self.bucket_name}/{s3_key} to {local_path}")

    def update_file(self, s3_key, data):
        self.write_json(s3_key, data)

    def delete_file(self, s3_key):
        self.s3.delete_object(Bucket=self.bucket_name, Key=s3_key)
        print(f"File s3://{self.bucket_name}/{s3_key} deleted.")

BUCKET_NAME = 'qanqa-prod'
aws_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
region_name = os.environ.get('REGION_NAME')

s3_manager = FileManager(aws_access_key=aws_access_key, aws_secret_key = aws_secret_key, region_name=region_name, bucket_name=BUCKET_NAME)

In [3]:
#Pydantic Models.
from enum import Enum
from typing import Dict
from pydantic import BaseModel, Field
from datetime import datetime

#User Library defined Data
class CourseMetadata(BaseModel):
    description: str = Field("default", description="Description of the course, e.g., 'Introduction to Python'")
    state: str = Field("default", description="Current state of the course, e.g., 'To evaluate'")
    level: str = Field("default", description="The level of the course, e.g., 'Beginning'")
    current_grade: str = Field("default", description="Current grade in the course, e.g., 'A'")
    goal: str = Field("default", description="Goal for the course, e.g., 'Understand the concepts'")
    progress: str = Field("default", description="Progress in the course as a percentage, e.g., '30%'")
    required_effort: str = Field("default", description="Effort required, e.g., 'Easy'")
    created_at: datetime = Field(..., description="Timestamp when the course was created")

class UserLibraryModel(BaseModel):
    courses: Dict[str, CourseMetadata] = Field(..., description="A dictionary of courses with course names as keys")

# Define Enum for states
class PodcastState(Enum):
    NOT_AVAILABLE = "Not Available"
    AVAILABLE = "Available"
    LISTENED = "Listened"

class AssessmentState(Enum):
    NOT_AVAILABLE = "Not Available"
    AVAILABLE = "Available"
    TAKEN = "Taken"

class EvaluationState(Enum):
    EVALUATED = "Evaluated"
    NOT_AVAILABLE = "Not Evaluated"

    APPROVED = "Approved"
    NOT_APPROVED = "Not Approved"

#Course defined Data
class LessonMetadata(BaseModel):
    podcast_state: str = Field(..., description="State of the podcast, e.g., 'Listened'")
    assessment_state: str = Field(..., description="State of the assessment, e.g., 'Taken'")
    evaluation_state: str = Field(..., description="State of the evaluation, e.g., 'Evaluated'")
    grade: str = Field(..., description="Grade for the lesson, e.g., 'A'")
    created_at: datetime = Field(..., description="Timestamp when the lesson was created")
    last_generated_assessment: datetime = Field(..., description="Timestamp for the last generated assessment")
    n_iteration: int = Field(..., description="Number of iterations")
    document_key_path: str = Field(..., description="S3 path for the document key")
    podcast_key_path: str = Field(..., description="S3 path for the podcast key")
    assessment_key_path: str = Field(..., description="S3 path for the assessment key")
    evaluation_key_path: str = Field(..., description="S3 path for the evaluation key")

class CourseModel(BaseModel):
    lessons: Dict[str, LessonMetadata] = Field(..., description="A dictionary of lessons with lesson names as keys")

#Lesson defined Data:
class LessonModel(BaseModel):
    lesson_path: str = Field(..., description="S3 path for the lesson")
    knowledge_graph_path: str = Field(..., description="S3 path for the knowledge graph")
    latest_version: int = Field(..., description="The latest version number of the lesson")
    assessments_params: str = Field(..., description="Assessment parameters, e.g., 'to be implemented'")
    created_at: datetime = Field(..., description="Timestamp when the lesson was created")
    updated_at: datetime = Field(..., description="Timestamp when the lesson was last updated")

Initial User Workflow:

1. Create Course (Add a Folder and create first metadata.json)
2. Add a Lesson
3. Generate Graph [Edit tracing is included]
4. Generate Podcast
5. Generate Assessment
6. Evaluate Assessment

------ Begin with the [Next Iteration] ------

While Student keeps evaluating:

7. Generate New Assessment
8. Evaluate New Assessment

------ Grading sufficient: Augment the difficult level. ------

9. Reach new Blooms Taxonomy level

In [4]:
username='robert'
from typing import List, Dict
from collections import defaultdict

class MetadataManager:
    def __init__(self, file_manager, base_key, level):
        self.file_manager = file_manager
        self.base_key = base_key.rstrip("/")  #bucket_name/user_name or bucket_name/user_name/course
        self.metadata_key = f"{self.base_key}/metadata.json"
        self.bucket_name = self.base_key.split('/')[0]
        self.username = self.base_key.split('/')[1]
        self.level = level
        self.path = self.username + '/' + self.base_key.split('/')[2] if self.level == 'course' else self.username

    def _parse_metadata(self):
        self.payload = self.file_manager.s3.list_objects_v2(Bucket=self.bucket_name, Prefix=self.path)['Contents']
        if self.level == 'user':
            course_dates = defaultdict(list)
            for obj in self.payload:
                key_parts = obj['Key'].split('/')
                if len(key_parts) >= 3:
                    course_name = key_parts[1]
                    course_dates[course_name].append(obj['LastModified'])
            courses = {}

            for course, dates in course_dates.items():
                created_at = min(dates) if dates else datetime.utcnow()
                courses[course] = CourseMetadata(created_at=created_at)

            return UserLibraryModel(courses=courses).dict()

        elif self.level == 'course':
            lessons_dict = defaultdict(dict)
            for obj in self.payload:
                key = obj['Key']
                key_parts = key.split('/')
                # Ensure the key has at least three parts: example_user/username/lesson_name/...
                if len(key_parts) >= 3:
                    lesson_name = key_parts[2]
                    # Determine the file type based on the key
                    if len(key_parts) == 4:
                        file_name = key_parts[3]
                        if file_name == 'document.txt':
                            lessons_dict[lesson_name]['document_key_path'] = f"s3://{self.bucket_name}/{key}"
                        elif file_name == 'podcast.mp3':
                            lessons_dict[lesson_name]['podcast_key_path'] = f"s3://{self.bucket_name}/{key}"
                        elif file_name == 'knowledge_graph.json':
                            lessons_dict[lesson_name]['knowledge_graph_key_path'] = f"s3://{self.bucket_name}/{key}"
                    elif len(key_parts) >= 5:
                        # Handle nested paths, e.g., Chapter5/v0/assessments.json
                        subfolder = key_parts[3]
                        file_name = key_parts[4]
                        if subfolder.startswith('v'):
                            # Assuming 'v0', 'v1', etc., represent iterations
                            iteration_number = int(subfolder[1:])  # Extract the number from 'v0'
                            lessons_dict[lesson_name]['n_iteration'] = max(
                                lessons_dict[lesson_name].get('n_iteration', 0),
                                iteration_number + 1  # Assuming iterations start at 0
                            )
                        if file_name == 'assessments.json':
                            lessons_dict[lesson_name]['assessment_key_path'] = f"s3://{self.bucket_name}/{key}"
                        elif file_name == 'evaluations.json':
                            lessons_dict[lesson_name]['evaluation_key_path'] = f"s3://{self.bucket_name}/{key}"

            lessons_metadata = {}
            for lesson_name, paths in lessons_dict.items():
                # Initialize default states
                podcast_state = PodcastState.NOT_AVAILABLE
                assessment_state = AssessmentState.NOT_AVAILABLE
                evaluation_state = EvaluationState.NOT_AVAILABLE
                grade = "Default"
                default_date = datetime(2023, 1, 1)

                # Determine states based on presence of keys
                if 'podcast_key_path' in paths:
                    podcast_state = PodcastState.LISTENED  # Or add logic to verify listening
                if 'assessment_key_path' in paths:
                    assessment_state = AssessmentState.TAKEN
                if 'evaluation_key_path' in paths:
                    evaluation_state = EvaluationState.EVALUATED
                # Extract metadata from knowledge_graph.json if available
                if 'knowledge_graph_key_path' in paths:
                    try:
                        knowledge_graph = {"grade": "Default"}
                        grade = "Default" #knowledge_graph.get('grade')
                    except Exception as e:
                        print(f"Error parsing knowledge_graph.json for lesson {lesson_name}: {e}")

                created_at_str = None
                last_generated_assessment_str = None
                last_generated_assessment = datetime.fromisoformat(last_generated_assessment_str) if last_generated_assessment_str else default_date
                created_at = datetime.fromisoformat(created_at_str) if created_at_str else default_date
                lesson_metadata = LessonMetadata(
                    podcast_state=podcast_state.value,
                    assessment_state=assessment_state.value,
                    evaluation_state=evaluation_state.value,
                    grade=grade,
                    created_at=created_at,
                    last_generated_assessment=last_generated_assessment,
                    n_iteration=paths.get('n_iteration', 0),
                    document_key_path=paths.get('document_key_path', ""),
                    podcast_key_path=paths.get('podcast_key_path', ""),
                    assessment_key_path=paths.get('assessment_key_path', ""),
                    evaluation_key_path=paths.get('evaluation_key_path',""),
                )
                lessons_metadata[lesson_name] = lesson_metadata

            return CourseModel(lessons=lessons_metadata).dict()

    def push_metadata(self):
        metadata = self._parse_metadata()
        
        self.file_manager.upload_file(obj=metadata, s3_key=self.path + "/metadata.json")
        print("data pushed to s3", self.path)
        return True
    
    def update_and_push(self):
        metadata = self._parse_metadata()

        if self.level == 'user':
            pass
        elif self.level == 'course':
            pass


        return 0

s3_manager = FileManager(aws_access_key=aws_access_key, aws_secret_key = aws_secret_key, region_name=region_name, bucket_name=BUCKET_NAME)
#metadata_user_manager = Metadata(s3_manager, f"qanqa-prod/{username}", UserLibraryModel)
#metadata_course_manager = Metadata(s3_manager, f"qanqa-prod/{username}", CourseModel)
#metadata_lesson_manager = Metadata()

s3_manager.upload_file({}, "pablo/metadata.json")

File uploaded to s3://qanqa-prod/pablo/metadata.json


In [6]:
s3_manager.check_if_file_exists("example_user/AINews/News on 2024/knowledge_graph.json")

True

In [104]:
metadata = MetadataManager(s3_manager, 'qanqa-prod/example_user', 'user')._parse_metadata()
metadata['courses']['AINews']['description'] = 'This is a course on AI News'
metadata['courses']['MatteoPasquinelli']['description'] = 'This is a course on Matteo Pasquinelli'
metadata

{'courses': {'AINews': {'description': 'This is a course on AI News',
   'state': 'default',
   'level': 'default',
   'current_grade': 'default',
   'goal': 'default',
   'progress': 'default',
   'required_effort': 'default',
   'created_at': datetime.datetime(2025, 1, 5, 17, 49, 36, tzinfo=tzutc())},
  'MatteoPasquinelli': {'description': 'This is a course on Matteo Pasquinelli',
   'state': 'default',
   'level': 'default',
   'current_grade': 'default',
   'goal': 'default',
   'progress': 'default',
   'required_effort': 'default',
   'created_at': datetime.datetime(2025, 1, 5, 17, 49, 34, tzinfo=tzutc())},
  'World History I': {'description': 'default',
   'state': 'default',
   'level': 'default',
   'current_grade': 'default',
   'goal': 'default',
   'progress': 'default',
   'required_effort': 'default',
   'created_at': datetime.datetime(2025, 1, 6, 16, 37, 28, tzinfo=tzutc())}}}

In [12]:
manager = MetadataManager(s3_manager, 'qanqa-prod/example_user/World History I', 'course')
metadata = manager._parse_metadata()
metadata

{'lessons': {'Coerced and SemiCoerced labor': {'podcast_state': 'Not Available',
   'assessment_state': 'Not Available',
   'evaluation_state': 'Not Evaluated',
   'grade': 'Default',
   'created_at': datetime.datetime(2023, 1, 1, 0, 0),
   'last_generated_assessment': datetime.datetime(2023, 1, 1, 0, 0),
   'n_iteration': 0,
   'document_key_path': 's3://qanqa-prod/example_user/World History I/Coerced and SemiCoerced labor/document.txt',
   'podcast_key_path': '',
   'assessment_key_path': '',
   'evaluation_key_path': ''},
  'Innovations and Inventions': {'podcast_state': 'Not Available',
   'assessment_state': 'Not Available',
   'evaluation_state': 'Not Evaluated',
   'grade': 'Default',
   'created_at': datetime.datetime(2023, 1, 1, 0, 0),
   'last_generated_assessment': datetime.datetime(2023, 1, 1, 0, 0),
   'n_iteration': 0,
   'document_key_path': 's3://qanqa-prod/example_user/World History I/Innovations and Inventions/document.txt',
   'podcast_key_path': '',
   'assessment_k

In [47]:
course_metadata_manager = MetadataManager(s3_manager, 'qanqa-prod/example_user/AI Engineering', 'course')
course_metadata = course_metadata_manager._parse_metadata()


In [52]:
for key, value in course_metadata['lessons'].items():
     if value.get('podcast_state','') != 'Not Available':
        print(key, value)

UserFeedback {'podcast_state': 'Listened', 'assessment_state': 'Not Available', 'evaluation_state': 'Not Evaluated', 'grade': 'Default', 'created_at': datetime.datetime(2023, 1, 1, 0, 0), 'last_generated_assessment': datetime.datetime(2023, 1, 1, 0, 0), 'n_iteration': 0, 'document_key_path': 's3://qanqa-prod/example_user/AI Engineering/UserFeedback/document.txt', 'podcast_key_path': 's3://qanqa-prod/example_user/AI Engineering/UserFeedback/podcast.mp3', 'assessment_key_path': '', 'evaluation_key_path': ''}


In [38]:
d = course_metadata['lessons']

from datetime import datetime

from datetime import datetime

from datetime import datetime

def parse_audio_files(data):
    audio_files = []
    # Loop over each key-value pair in the provided JSON-like dict
    for key, value in data.items():
        # Check if there's a non-empty podcast key path indicating an audio file
        if value.get('podcast_key_path'):
            # Construct audio file information using available data or hard-coded defaults
            duration = '3m 40s'  # Hard-coded duration as placeholder
            file_id = '1'        # Hard-coded id as placeholder
            filename = f"{key} - Podcast"  # Create filename using the key
            # Format created_at date if available, else use a default date
            created_at_obj = value.get('created_at')
            if isinstance(created_at_obj, datetime):
                created_at = created_at_obj.strftime('%Y-%m-%d')
            else:
                created_at = '2025-01-01'  # Default date if not provided

            # Append constructed audio file dict to the list
            audio_files.append({
                'duration': duration,
                'id': file_id,
                'filename': filename,
                'created_at': created_at
            })
            # Only one match needed, so break after finding the first
            break

    return audio_files

audop_files = parse_audio_files(d)

print(audop_files)

[{'duration': '3m 40s', 'id': '1', 'filename': 'UserFeedback - Podcast', 'created_at': '2023-01-01'}]


In [44]:
metadata

{'lessons': {'Coerced and SemiCoerced labor': {'podcast_state': 'Not Available',
   'assessment_state': 'Not Available',
   'evaluation_state': 'Not Evaluated',
   'grade': 'Default',
   'created_at': datetime.datetime(2023, 1, 1, 0, 0),
   'last_generated_assessment': datetime.datetime(2023, 1, 1, 0, 0),
   'n_iteration': 0,
   'document_key_path': 's3://qanqa-prod/example_user/World History I/Coerced and SemiCoerced labor/document.txt',
   'podcast_key_path': '',
   'assessment_key_path': '',
   'evaluation_key_path': ''},
  'Innovations and Inventions': {'podcast_state': 'Not Available',
   'assessment_state': 'Not Available',
   'evaluation_state': 'Not Evaluated',
   'grade': 'Default',
   'created_at': datetime.datetime(2023, 1, 1, 0, 0),
   'last_generated_assessment': datetime.datetime(2023, 1, 1, 0, 0),
   'n_iteration': 0,
   'document_key_path': 's3://qanqa-prod/example_user/World History I/Innovations and Inventions/document.txt',
   'podcast_key_path': '',
   'assessment_k

In [106]:
course_metadata_manager = MetadataManager(s3_manager, 'qanqa-prod/example_user/World History I', 'course')._parse_metadata()
course_metadata_manager['lessons']

{'Coerced and SemiCoerced labor': {'podcast_state': 'Not Available',
  'assessment_state': 'Not Available',
  'evaluation_state': 'Not Evaluated',
  'grade': 'Default',
  'created_at': datetime.datetime(2023, 1, 1, 0, 0),
  'last_generated_assessment': datetime.datetime(2023, 1, 1, 0, 0),
  'n_iteration': 0,
  'document_key_path': 's3://qanqa-prod/example_user/World History I/Coerced and SemiCoerced labor/document.txt',
  'podcast_key_path': '',
  'assessment_key_path': '',
  'evaluation_key_path': ''},
 'Innovations and Inventions': {'podcast_state': 'Not Available',
  'assessment_state': 'Not Available',
  'evaluation_state': 'Not Evaluated',
  'grade': 'Default',
  'created_at': datetime.datetime(2023, 1, 1, 0, 0),
  'last_generated_assessment': datetime.datetime(2023, 1, 1, 0, 0),
  'n_iteration': 0,
  'document_key_path': 's3://qanqa-prod/example_user/World History I/Innovations and Inventions/document.txt',
  'podcast_key_path': '',
  'assessment_key_path': '',
  'evaluation_key_

In [109]:
assessment = s3_manager.read_json(s3_key=f"example_user/AINews/News on 2024/v0/assessments.json")

In [89]:
docs_metadata = s3_manager.read_json(s3_key=f"example_user/metadata.json")['courses']

for id, docs in docs_metadata.items():
    print(id, docs)

AINews {'state': 'default', 'level': 'default', 'current_grade': 'default', 'goal': 'default', 'progress': 'default', 'required_effort': 'default', 'created_at': '2025-01-05 17:49:36+00:00'}
MatteoPasquinelli {'state': 'default', 'level': 'default', 'current_grade': 'default', 'goal': 'default', 'progress': 'default', 'required_effort': 'default', 'created_at': '2025-01-05 17:49:34+00:00'}


In [ ]:
s3_manager.upload_file({}, "example_user/metadata.json")

{'AINews': {'state': 'default',
  'level': 'default',
  'current_grade': 'default',
  'goal': 'default',
  'progress': 'default',
  'required_effort': 'default',
  'created_at': '2025-01-05 17:49:36+00:00'},
 'MatteoPasquinelli': {'state': 'default',
  'level': 'default',
  'current_grade': 'default',
  'goal': 'default',
  'progress': 'default',
  'required_effort': 'default',
  'created_at': '2025-01-05 17:49:34+00:00'}}

In [80]:
username = 'pablo'
docs_metadata = s3_manager.read_json(s3_key=f"{username}/metadata.json")
docs_metadata

{}

In [82]:
#Create Course (Add a Folder and create first metadata.json)
from datetime import datetime
username = 'pablo-test'

course_form = {
    "course_name": "Course Test name",
    "course_goal": "Course Test Goal",
    "required_effort": "easy",
    "created_at" : datetime.now().strftime(format="%Y-%m-%d-%H:%M")
}

s3_manager.upload_file({}, f"{username}/metadata.json")
s3_manager.upload_file({}, f"{username}/{course_form["course_name"]}/metadata.json")

File uploaded to s3://qanqa-prod/pablo-test/metadata.json
File uploaded to s3://qanqa-prod/pablo-test/Course Test name/metadata.json


In [83]:
user_metadata_manager = MetadataManager(s3_manager, 'qanqa-prod/example_user/', level= 'user')
_ = user_metadata_manager.push_metadata()
_

File uploaded to s3://qanqa-prod/example_user/metadata.json
data pushed to s3 example_user


True

In [84]:
docs_metadata = s3_manager.read_json(s3_key="example_user/metadata.json")
docs_metadata

{'courses': {'AINews': {'state': 'default',
   'level': 'default',
   'current_grade': 'default',
   'goal': 'default',
   'progress': 'default',
   'required_effort': 'default',
   'created_at': '2025-01-05 17:49:36+00:00'},
  'MatteoPasquinelli': {'state': 'default',
   'level': 'default',
   'current_grade': 'default',
   'goal': 'default',
   'progress': 'default',
   'required_effort': 'default',
   'created_at': '2025-01-05 17:49:34+00:00'}}}

### Metadata Manager

#### Domain Classes (Library, Course, Lesson)

In [ ]:
from enum import Enum

class PodcastState(Enum):
    NOT_AVAILABLE = "Not Available"
    AVAILABLE = "Available"
    LISTENED = "Listened"

class AssessmentState(Enum):
    NOT_AVAILABLE = "Not Available"
    AVAILABLE = "Available"
    TAKED = "Taked"

class GradeState(Enum):
    EVALUATED = "Evaluated"
    NOT_EVALUATED = "Not Evaluated"

    APPROVED = "Approved"
    NOT_APPROVED = "Not Approved"

In [ ]:
import uuid
from datetime import datetime

class Lesson:
    def __init__(self, title, document_url, lesson_id=None):
        self.lesson_id = lesson_id or str(uuid.uuid4())
        self.title = title
        self.document_url = document_url
        self.podcast_state = PodcastState.NOT_AVAILABLE
        self.assessment_state = AssessmentState.NOT_AVAILABLE
        self.last_edited_time = datetime.now()

    def to_dict(self):
        """
        Convert the Lesson to a dictionary for JSON serialization
        (which will be stored in metadata.json).
        """
        return {
            "lesson_id": self.lesson_id,
            "title": self.title,
            "document_url": self.document_url,
            "podcast_state": self.podcast_state.value,
            "assessment_state": self.assessment_state.value,
            "last_edited_time": str(self.last_edited_time)
        }

    @classmethod
    def from_dict(cls, data_dict):
        """
        Create a Lesson instance from a dictionary loaded from S3.
        """
        lesson = cls(
            title=data_dict["title"],
            document_url=data_dict["document_url"],
            lesson_id=data_dict["lesson_id"]
        )
        # Restore states
        lesson.podcast_state = PodcastState(data_dict["podcast_state"])
        lesson.assessment_state = AssessmentState(data_dict["assessment_state"])
        lesson.last_edited_time = datetime.fromisoformat(data_dict["last_edited_time"])
        return lesson

In [ ]:
class Course:
    def __init__(self, name, description, course_id=None):
        self.course_id = course_id or str(uuid.uuid4())
        self.name = name
        self.description = description
        self.lessons = {}
        self.last_edited_time = datetime.now()

    def to_dict(self):
        return {
            "course_id": self.course_id,
            "name": self.name,
            "description": self.description,
            "last_edited_time": str(self.last_edited_time),
            "lessons": {
                lesson_id: lesson.to_dict() 
                for lesson_id, lesson in self.lessons.items()
            }
        }

    @classmethod
    def from_dict(cls, data_dict):
        course = cls(
            name=data_dict["name"],
            description=data_dict["description"],
            course_id=data_dict["course_id"]
        )
        course.last_edited_time = datetime.fromisoformat(data_dict["last_edited_time"])
        for lesson_id, lesson_data in data_dict["lessons"].items():
            lesson = Lesson.from_dict(lesson_data)
            course.lessons[lesson_id] = lesson
        return course

In [ ]:
class Library:
    def __init__(self, user_id):
        self.user_id = user_id
        self.courses = {}

    def to_dict(self):
        return {
            "user_id": self.user_id,
            "courses": {
                course_id: course.to_dict()
                for course_id, course in self.courses.items()
            }
        }

    @classmethod
    def from_dict(cls, data_dict):
        library = cls(user_id=data_dict["user_id"])
        for course_id, course_data in data_dict["courses"].items():
            course_obj = Course.from_dict(course_data)
            library.courses[course_id] = course_obj
        return library